In [1]:
import aocd
import dataclasses
import numpy as np
import enum

real_data = aocd.get_data(day=14, year=2022)
test_data = """498,4 -> 498,6 -> 496,6
503,4 -> 502,4 -> 502,9 -> 494,9"""

In [4]:
from typing import Sequence, Union, Tuple
import copy
import itertools

def draw_line(
    pos0: Tuple[int, int], 
    pos1: Tuple[int, int], 
    canvas: np.ndarray,
) -> np.ndarray:
    """Draws a line on the terrain.
    
    Args:
        pos0: position of the first point.
        pos1: position of the second point.
        canvas: input canvas.
        
    Returns:
        output canvas after the line is drawn.
    """
    if pos0[0] < pos1[0]:
        a, b = pos0[0], pos1[0] + 1
    else:
        a, b = pos1[0], pos0[0] + 1
        
    if pos0[1] < pos1[1]:
        c, d = pos0[1], pos1[1] + 1
    else:
        c, d = pos1[1], pos0[1] + 1
    
    canvas[a:b, c:d] = 1
    return copy.deepcopy(canvas.astype(int))
    
def draw_n_lines(
    line_input: str,
    canvas: np.ndarray,
) -> np.ndarray:
    """Draws multiple lines on the terrain, given a line of input.
    
    Args:
        line_input: the input line.
        canvas: input canvas.
        
    Returns:
        output canvas after the line is drawn.
    """
    positions = line_input.split(" -> ")
    for i in range(len(positions) - 1):
        pos0 = [int(x) for x in positions[i].split(",")]
        pos1 = [int(x) for x in positions[i + 1].split(",")]
        canvas = draw_line(pos0, pos1, canvas)
    return canvas
    
def next_frame(
    terrain: np.ndarray, 
    sand_pos: Tuple[int, int],
) -> Tuple[int, int]:
    """Simulates the next position of the sand particle.
    
    Args:
        terrain: the input terrain.
        sand_pos: the position of the sand particle.
        
    Returns:
        the position of the sand in the next frame.
    """
    # Define the three critical positions.
    down_pos = sand_pos[0], sand_pos[1] + 1
    down_left_pos = sand_pos[0] - 1, sand_pos[1] + 1
    down_right_pos = sand_pos[0] + 1, sand_pos[1] + 1
    
    if terrain[down_pos] == 0:
        return down_pos
    elif terrain[down_left_pos] == 0:
        return down_left_pos
    elif terrain[down_right_pos] == 0:
        return down_right_pos
    else:
        return copy.deepcopy(sand_pos)
    

def start_to_rest(
    terrain: np.ndarray, 
    sand_pos: Tuple[int, int],
    floor: int,
) -> Union[np.ndarray, None]:
    """Simulates from the start of a sand particle to when it rests.
    
    Args:
        terrain: the input terrain.
        sand_pos: the position of the sand particle.
        floor: the floor position. The 
        
    Returns:
        the new terrain with sand particle integrated as a rock.
        if the sand reaches the floor, then returns None.
    """
    while True:
        # Halt if the sand hits the floor.
        if sand_pos[1] >= floor:
            return None
        
        old_sand_pos = copy.deepcopy(sand_pos)
        sand_pos = next_frame(terrain, old_sand_pos)
        if sand_pos == old_sand_pos:
            break
    new_terrain = copy.deepcopy(terrain)
    new_terrain[sand_pos] = 1
    return new_terrain
    
def find_bottom_pos(terrain: np.ndarray) -> int:
    """Finds the bottom position.
    
    Args:
        terrain: the input terrain to scan for the bottom.
        
    Returns:
        the bottom position.
    """
    pos = terrain.shape[1] - 1
    while True:
        if np.sum(terrain[:, pos]) != 0:
            return pos
        pos -= 1
    
@dataclasses.dataclass
class SolverA:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.splitlines()
        self.sand_pos = (500, 0)
        
    def debug(self):
        """For debugging."""
        terrain = np.zeros([600, 600])
        sand_pos = self.sand_pos
        for line in self.lines:
            terrain = draw_n_lines(line, terrain)

        bottom_pos = find_bottom_pos(terrain, 4)
        run_n_particles = 25
        for n_particle in range(run_n_particles):
            terrain = start_to_rest(terrain, sand_pos, bottom_pos)     
            if terrain is None:
                return n_particle
        print(np.transpose(terrain)[0:10, 494:504])
        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        terrain = np.zeros([600, 600])
        sand_pos = self.sand_pos
        for line in self.lines:
            terrain = draw_n_lines(line, terrain)

        bottom_pos = find_bottom_pos(terrain)
        
        n_particle = 0
        while True:
            terrain = start_to_rest(terrain, sand_pos, bottom_pos)     
            if terrain is None:
                return n_particle
            n_particle += 1

In [5]:
SolverA(test_data).find_answer()

24

In [6]:
answer = SolverA(real_data).find_answer()
aocd.submit(answer, part="a", day=14, year=2022)

Part a already solved with same answer: 696


In [7]:
@dataclasses.dataclass
class SolverB:
    """
    A solver instance.
    
    args:
        raw_data: the raw input data.
    """
    raw_data: str

    def __post_init__(self):
        self.lines = self.raw_data.splitlines()
        self.sand_pos = (500, 0)
        
    def find_answer(self) -> int:
        """Finds the answer.
        
        Returns:
            The answer.
        """
        terrain = np.zeros([1200, 600])
        sand_pos = self.sand_pos
        for line in self.lines:
            terrain = draw_n_lines(line, terrain)

        bottom_pos = find_bottom_pos(terrain)
        terrain[:, bottom_pos + 2] = 1
        
        n_particle = 0
        while True:
            old_terrain = copy.copy(terrain)
            terrain = start_to_rest(old_terrain, sand_pos, np.Inf) 
            if np.all(old_terrain == terrain):
                return n_particle
            n_particle += 1

In [8]:
SolverB(test_data).find_answer()

93

In [9]:
answer = SolverB(real_data).find_answer()
aocd.submit(answer, part="b", day=14, year=2022)

Part b already solved with same answer: 23610
